# Задание №2. Парсинг HH.ru + предобработка текстовых данных
Используя публичный API сайта HeadHunter (особое внимание на метод https://api.hh.ru/vacancies/<id>) необходимо:

- скачать минимум 200 вакансий по заданному ключевому слову (можете задать слова, характеризующие профессии в IT); если вакансий менее 200 - скачать все, что есть;

- предобработать каждую вакансию (очистить от тегов и специальных символов, токенизировать, нормализовать любым из двух способов, удалить стоп-слова);

- сохранить все полученные вакансии (список списков токенов) на диск (Google Drive) с помощью pickle.

In [ ]:
import requests
import pandas as pd
from torch import nn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from pymorphy3 import MorphAnalyzer
import string
import numpy as np